In [1]:
!pip install -r requirement.txt

In [2]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim import corpora, models
from gensim.utils import simple_preprocess
from nltk import WordNetLemmatizer
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pyLDAvis.gensim_models
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oyuesan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/oyuesan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/oyuesan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
# Read the data from the file 'DH_CollectingData2022_review.tsv.txt'
df = pd.read_csv('DH_CollectingData2022_review.tsv', sep='\t', header=None, names=['sentence', 'sentiment'],
                 encoding='utf-8')
# Create a set of stop words in English
stop_words = set(stopwords.words('english'))
# Create a WordNetLemmatizer object to perform lemmatization
lemmatizer = WordNetLemmatizer()


# Preprocess function
def preprocess(text):
    # Convert to lowercase
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    # Tokenize text using simple_preprocess
    tokens = simple_preprocess(text, deacc=True)
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatize words
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join tokens back into string
    return tokens

# Define a function named get_num that takes a row of data as input
def get_num(row):
    # Extract the sentence and sentiment values from the row
    sentence = row['sentence']
    number = row['sentiment']
    # If the sentiment value is not NaN, return it
    if pd.notna(row['sentiment']):
        return number
    # The bug fix for file.
    if sentence and sentence[-1].isdigit():
        # If the last character of the sentence is a digit, extract the number.
        if len(sentence) > 1 and sentence[-2] == '-':
            return int(sentence[-1])
        # Otherwise, the number is positive
        else:
            return -int(sentence[-1])
    # If the sentence doesn't end with a digit, return NaN
    else:
        return pd.NA

# Apply get_num to 'sentiment' column
df['sentiment'] = df.apply(get_num, axis=1)
# Apply preprocessing to 'sentence' column
df['clean_text'] = df['sentence'].apply(preprocess)

In [4]:
df.head()

,sentence,sentiment,clean_text
0,"For Nik, he only wants to silence the cacophon...",0.0,"[nik, want, silence, cacophony, sound, color, ..."
1,I can play this two ways\t0,0,"[play, two, way]"
2,"Mild, because it isn't conclusive, and doesn't...",-1.0,"[mild, isnt, conclusive, doesnt, give, u, info..."
3,You can also get some more information about t...,0.0,"[also, get, information, book, writing, exclus..."
4,"Soon, Hero, who has never had friends, is thru...",0.0,"[soon, hero, never, friend, thrust, school, qu..."


In [5]:
# Get all sentences from the 'clean_text' column of the DataFrame and create a list
all_sentences = df['clean_text'].tolist()
# Create a TF-IDF matrix using the TfidfVectorizer from scikit-learn
vectorizer = TfidfVectorizer(stop_words='english', max_features=20)
tfidf_matrix = vectorizer.fit_transform(df['sentence'])
# Extract the top 20 keywords from the TF-IDF matrix
# First, sort the feature weights in descending order
indices = np.argsort(vectorizer.idf_)[::-1]
# Get the feature names from the vectorizer
features = vectorizer.get_feature_names_out()
# Set the number of top features to extract
top_n = 20
# Create a list of the top n features
top_features = [features[i] for i in indices[:top_n]]
# Print the top features
print("Top 20 keywords: ", top_features)

Top 20 keywords:  ['way', 'little', 'mystery', 'did', 'interesting', 'world', 'reading', 'think', 'character', 'great', 'novel', 'love', 'really', 'like', 'good', 'characters', 'just', 'story', 'read', 'book']


In [6]:
# Create a new DataFrame containing only the rows where the 'sentiment' column equals 1 (positive sentiment)
positive_df = df[df['sentiment'] == 1]
# Create a new DataFrame containing only the rows where the 'sentiment' column equals -1 (negative sentiment)
negative_df = df[df['sentiment'] == -1]

In [7]:
# Get the 'clean_text' column from the positive and negative DataFrames
data_words_pos = positive_df['clean_text']
data_words_neg = negative_df['clean_text']
# Define the range of hyperparameters to test
num_topics_range = [3, 5, 7]
num_passes_range = [5, 10, 15]
chunksize_range = [3, 5, 7, 9]
# Define a function to calculate the coherence score for a given model
def compute_coherence(model, corpus, texts):
    coherence_model = CoherenceModel(model=model, corpus=corpus, texts=texts, coherence='c_v')
    return coherence_model.get_coherence()
# Initialize variables to store the best hyperparameters and resulting model
def find_best_hyperparameter(texts):
    best_coherence = 0
    best_num_topics = None
    best_num_passes = None
    best_chunksize = None
    best_model = None
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    # Perform grid search
    for i, num_topics in enumerate(num_topics_range):
        for j, num_passes in enumerate(num_passes_range):
            for k, chunksize in enumerate(chunksize_range):
                # Fit the LDA model using the current hyperparameters
                lda_model = gensim.models.ldamodel.LdaModel(
                    corpus=corpus, id2word=dictionary, num_topics=num_topics,
                    random_state=42, update_every=1, chunksize=chunksize,
                    passes=num_passes, alpha='auto', per_word_topics=True
                )
                # Compute coherence score for the resulting model
                coherence = compute_coherence(lda_model, corpus, texts)
                # Update the best hyperparameters and resulting model if coherence score is higher
                if coherence > best_coherence:
                    best_coherence = coherence
                    best_num_topics = num_topics
                    best_num_passes = num_passes
                    best_chunksize = chunksize
                    best_model = lda_model
    return best_coherence, best_num_topics, best_num_passes, best_chunksize, best_model, corpus, dictionary



In [8]:
# Print the best hyperparameters and resulting model
best_coherence, best_num_topics, best_num_passes, best_chunksize, best_model, corpus, dictionary = find_best_hyperparameter(data_words_pos)
# Print the best hyperparameters and coherence score
print('Best coherence:', best_coherence)
print('Best num topics:', best_num_topics)
print('Best num passes:', best_num_passes)
print('Best chunksize:', best_chunksize)
# Print the top topics and their associated keywords for the resulting model
print(best_model.print_topics())

Best coherence: 0.6117331561233724
Best num topics: 5
Best num passes: 15
Best chunksize: 3
[(0, '0.066*"read" + 0.021*"voice" + 0.016*"see" + 0.015*"mystery" + 0.015*"fun" + 0.015*"liked" + 0.011*"great" + 0.011*"kept" + 0.010*"also" + 0.010*"story"'), (1, '0.022*"bit" + 0.021*"im" + 0.021*"new" + 0.020*"hill" + 0.019*"really" + 0.018*"take" + 0.017*"thriller" + 0.014*"little" + 0.014*"coming" + 0.012*"perfect"'), (2, '0.048*"character" + 0.025*"much" + 0.022*"took" + 0.021*"lot" + 0.018*"reader" + 0.015*"better" + 0.014*"think" + 0.012*"way" + 0.010*"get" + 0.010*"part"'), (3, '0.068*"book" + 0.056*"well" + 0.023*"interesting" + 0.020*"time" + 0.013*"crawford" + 0.011*"something" + 0.010*"worth" + 0.010*"evil" + 0.008*"he" + 0.008*"tale"'), (4, '0.027*"good" + 0.024*"novel" + 0.023*"star" + 0.021*"writer" + 0.012*"beautiful" + 0.012*"enjoy" + 0.010*"first" + 0.009*"life" + 0.008*"every" + 0.008*"make"')]


In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(best_model, corpus, dictionary)
vis

/Users/oyuesan/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.037640 -0.074519       1        1  23.034890
2     -0.025259  0.062994       2        1  21.179990
3      0.100350 -0.007664       3        1  20.020463
1     -0.024585  0.016881       4        1  18.271386
4     -0.012865  0.002308       5        1  17.493271, topic_info=          Term       Freq      Total Category  logprob  loglift
13        read  24.000000  24.000000  Default  30.0000  30.0000
0         book  21.000000  21.000000  Default  29.0000  29.0000
37        well  18.000000  18.000000  Default  28.0000  28.0000
16   character  16.000000  16.000000  Default  27.0000  27.0000
46        good   7.000000   7.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
106   remained   0.967969   5.023345   Topic5  -5.6371   0.0967
711      didnt   1.142204   7.738543   Topic5  -5.4716  -0.1699
762    looking   1.042605   6.534328   Topic5  -5.5629  -0.0920
232      three   0.949961   5.040216   Topic5  -5.6559   0.0746
665       work   1.021229   6.469175   Topic5  -5.5836  -0.1027

[241 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
843       1  0.222243  abbott
843       2  0.222243  abbott
843       3  0.222243  abbott
843       4  0.222243  abbott
843       5  0.222243  abbott
...     ...       ...     ...
28        1  0.272094     yet
28        2  0.272094     yet
28        3  0.272094     yet
28        4  0.272094     yet
28        5  0.272094     yet

[306 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 4, 2, 5])

In [10]:
# Call the find_best_hyperparameter function with the 'data_words_neg' variable as input to get the best hyperparameters and resulting model
best_coherence, best_num_topics, best_num_passes, best_chunksize, best_model, corpus, dictionary = find_best_hyperparameter(data_words_neg)
# Print the best hyperparameters and coherence score
print('Best coherence:', best_coherence)
print('Best num topics:', best_num_topics)
print('Best num passes:', best_num_passes)
print('Best chunksize:', best_chunksize)
# Print the top topics and their associated keywords for the resulting model
print(best_model.print_topics())

Best coherence: 0.5279284985362422
Best num topics: 3
Best num passes: 5
Best chunksize: 5
[(0, '0.032*"little" + 0.025*"unfortunately" + 0.025*"cast" + 0.025*"justice" + 0.025*"killer" + 0.018*"felt" + 0.018*"like" + 0.014*"got" + 0.014*"time" + 0.013*"much"'), (1, '0.019*"feel" + 0.019*"get" + 0.016*"conclusion" + 0.016*"got" + 0.014*"character" + 0.014*"didnt" + 0.012*"even" + 0.012*"ethan" + 0.011*"interesting" + 0.010*"reading"'), (2, '0.040*"book" + 0.028*"first" + 0.025*"writing" + 0.023*"part" + 0.022*"sequel" + 0.020*"pevels" + 0.020*"revelation" + 0.020*"ignored" + 0.020*"flaw" + 0.020*"certain"')]


In [11]:
vis = pyLDAvis.gensim_models.prepare(best_model, corpus, dictionary)
vis

/Users/oyuesan/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.152806 -0.026859       1        1  34.259306
0      0.115155 -0.066002       2        1  34.166041
1      0.037651  0.092861       3        1  31.574653, topic_info=              Term      Freq     Total Category  logprob  loglift
13            book  8.000000  8.000000  Default  30.0000  30.0000
377         little  6.000000  6.000000  Default  29.0000  29.0000
17           first  5.000000  5.000000  Default  28.0000  28.0000
25   unfortunately  5.000000  5.000000  Default  27.0000  27.0000
390         killer  5.000000  5.000000  Default  26.0000  26.0000
..             ...       ...       ...      ...      ...      ...
366           mind  1.544127  2.249191   Topic3  -4.7513   0.7767
295           even  2.113840  3.386081   Topic3  -4.4373   0.6816
316            got  2.863850  5.774176   Topic3  -4.1336   0.4516
63     interesting  2.013203  3.540928   Topic3  -4.4861   0.5882
122      character  2.544131  6.013993   Topic3  -4.2520   0.2925

[135 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
321       2  0.594458        absence
299       3  0.625650       attached
13        1  0.965272           book
305       3  0.631899           cait
388       2  0.950813           cast
...     ...       ...            ...
340       2  0.759398           time
25        2  0.938430  unfortunately
187       2  0.550075          wasnt
375       1  0.929443        writing
214       3  0.680558             xd

[112 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])